In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def load_moex_data(security='GAZP', years=5):
    """
    Загружает исторические данные с MOEX для указанной ценной бумаги.
    
    Parameters:
    -----------
    security : str
        Тикер ценной бумаги (например, 'GAZP' для Газпрома)
    years : int
        Количество лет истории для загрузки
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame с историческими данными, содержащий следующие колонки:
        - TRADEDATE: дата торгов
        - SECID: идентификатор ценной бумаги
        - OPEN: цена открытия
        - HIGH: максимальная цена
        - LOW: минимальная цена
        - CLOSE: цена закрытия
        - VOLUME: объем торгов в лотах
        - WAPRICE: средневзвешенная цена
    """
    # Формируем даты начала и конца периода
    end_date = datetime.now().date()
    start_date = end_date - timedelta(days=365 * years)
    
    # Базовый URL ISS API для исторических данных
    base_url = "https://iss.moex.com/iss/history/engines/stock/markets/shares/securities"
    
    # Параметры запроса
    params = {
        'from': start_date.strftime('%Y-%m-%d'),
        'till': end_date.strftime('%Y-%m-%d'),
        'start': 0
    }
    
    # Список для накопления данных
    all_data = []
    
    while True:
        # Формируем URL для запроса
        url = f"{base_url}/{security}.json"
        
        # Делаем запрос к API
        r = requests.get(url, params=params)
        r.raise_for_status()  # Проверяем на ошибки
        data = r.json()
        
        # Получаем данные из секции 'history'
        history_data = data.get('history', {}).get('data', [])
        if not history_data:
            break  # Нет данных или достигнут конец
        
        all_data.extend(history_data)
        
        # Проверяем, есть ли еще данные
        cursor = data.get('history.cursor', {}).get('data', [])
        if not cursor:
            break
        
        # Получаем информацию о пагинации
        total_rows = cursor[0][1]
        start = cursor[0][0] + cursor[0][2]  # current_start + page_size
        
        if start >= total_rows:
            break
            
        params['start'] = start
    
    # Создаем DataFrame
    columns = data.get('history', {}).get('columns', [])
    df = pd.DataFrame(all_data, columns=columns)
    
    # Фильтруем только по основному режиму торгов TQBR
    df = df[df['BOARDID'] == 'TQBR']
    
    # Оставляем только нужные колонки
    needed_columns = [
        'TRADEDATE',
        'SECID',
        'OPEN',
        'HIGH',
        'LOW',
        'CLOSE',
        'VOLUME',
        'WAPRICE'
    ]
    df = df[needed_columns]
    
    # Преобразуем даты и сортируем
    df['TRADEDATE'] = pd.to_datetime(df['TRADEDATE'])
    df = df.sort_values('TRADEDATE').reset_index(drop=True)
    
    # Проверяем на пропущенные значения
    if df.isnull().any().any():
        print(f"Warning: Found {df.isnull().sum().sum()} missing values in the data")
    
    return df

# Пример использования:
if __name__ == "__main__":
    # Загружаем данные для Газпрома за последние 5 лет
    df = load_moex_data('GAZP', 5)
    print(f"Loaded {len(df)} rows of data")
    print("\nFirst few rows:")
    print(df.head())
    print("\nData info:")
    print(df.info())